In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import wget

In [2]:
spark = SparkSession.builder\
    .master('local[*]')\
    .getOrCreate()

sc = spark.sparkContext

22/05/31 19:36:31 WARN Utils: Your hostname, PANZER resolves to a loopback address: 127.0.1.1; using 192.168.126.228 instead (on interface eth0)
22/05/31 19:36:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/31 19:36:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/31 19:36:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### RDD

In [3]:
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


In [4]:
print(sc.version)

3.2.1


In [5]:
lst = [25,90,81,37,776,3320]
rdd_list = sc.parallelize(lst)

In [6]:
rdd_list.count()

6

In [7]:
rdd_list.collect()

[25, 90, 81, 37, 776, 3320]

In [8]:
wget.download("https://archive.org/download/carros_202205/sentimentos.csv",".")

'./sentimentos (1).csv'

In [9]:
sentimentoRDD = sc.textFile("sentimentos.csv")

In [10]:
sentimentoRDD.count()

100

In [11]:
sentimentoRDD.take(5)

['positivo,Esse livro é incrível.',
 'positivo,Um dos melhores livros que eu já li.',
 'positivo,um dos melhores livros que eu já li',
 'positivo,Acho que ele tem um conteúdo que vai além do que está em sua descrição.',
 'positivo,O Sol é para todos é profundo e emocionante']

In [12]:
sentimentoRDD = sentimentoRDD.map(lambda x : x.upper())
sentimentoRDD.take(5)

['POSITIVO,ESSE LIVRO É INCRÍVEL.',
 'POSITIVO,UM DOS MELHORES LIVROS QUE EU JÁ LI.',
 'POSITIVO,UM DOS MELHORES LIVROS QUE EU JÁ LI',
 'POSITIVO,ACHO QUE ELE TEM UM CONTEÚDO QUE VAI ALÉM DO QUE ESTÁ EM SUA DESCRIÇÃO.',
 'POSITIVO,O SOL É PARA TODOS É PROFUNDO E EMOCIONANTE']

In [13]:
sentimentoRDD = sentimentoRDD.filter(lambda line : "SOL" in  line)
sentimentoRDD.take(5)

['POSITIVO,O SOL É PARA TODOS É PROFUNDO E EMOCIONANTE',
 'POSITIVO,O QUE FALAR DESDE LIVRO INCRÍVEL? O SOL É PARA TODOS ME CONQUISTOU DE UMA FORMA PURA! ',
 'NEGATIVO,SOL PARA TODOS É UM LIVRO RUIM',
 'NEGATIVO,O SOL PARA TODOS NÃO É PARA TODOS.',
 'NEGATIVO,AND I ABSOLUTELY LOATHE THE DA VINCI CODE.']

In [14]:
sentimentoRDD.map(lambda line: len(line.split())).reduce(lambda a,b: a if (a>b) else b)

17

In [15]:
def max(a,b):
    if a> b:
        return a
    return b

sentimentoRDD.map(lambda line: len(line.split())).reduce(max)

17

In [16]:
#contar cada palavra
contapalavras = (sentimentoRDD.flatMap(
    lambda line: line.split()
).map(lambda palavra: (palavra,1)
).reduceByKey(lambda a,b: a+b)
)

contapalavras.collect()

[('SOL', 3),
 ('É', 5),
 ('TODOS', 4),
 ('QUE', 1),
 ('LIVRO', 2),
 ('INCRÍVEL?', 1),
 ('O', 1),
 ('ME', 1),
 ('CONQUISTOU', 1),
 ('DE', 1),
 ('NEGATIVO,SOL', 1),
 ('UM', 1),
 ('RUIM', 1),
 ('NEGATIVO,O', 1),
 ('TODOS.', 1),
 ('NEGATIVO,AND', 1),
 ('LOATHE', 1),
 ('VINCI', 1),
 ('CODE.', 1),
 ('POSITIVO,O', 2),
 ('PARA', 5),
 ('PROFUNDO', 1),
 ('E', 1),
 ('EMOCIONANTE', 1),
 ('FALAR', 1),
 ('DESDE', 1),
 ('UMA', 1),
 ('FORMA', 1),
 ('PURA!', 1),
 ('NÃO', 1),
 ('I', 1),
 ('ABSOLUTELY', 1),
 ('THE', 1),
 ('DA', 1)]

#### RDD TRANSFORM/ACTION

In [17]:
lista = [124,901,652,102,397]

In [18]:
lstrdd = sc.parallelize(lista)

In [19]:
lstrdd.collect()


[124, 901, 652, 102, 397]

In [20]:
lstrdd.count()

5

In [21]:
wget.download('https://archive.org/download/carros_202205/carros.csv')

'carros (1).csv'

In [22]:
carro = sc.textFile('./carros.csv')

In [23]:
carro.first()

'MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE'

In [24]:
carro.take(5)

['MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE',
 'subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118',
 'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151',
 'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195',
 'toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348']

In [25]:
carro.cache()

./carros.csv MapPartitionsRDD[19] at textFile at NativeMethodAccessorImpl.java:0

In [26]:
tabcar = carro.map(lambda x: x.replace(',','\t'))
tabcar.take(5)

['MAKE\tFUELTYPE\tASPIRE\tDOORS\tBODY\tDRIVE\tCYLINDERS\tHP\tRPM\tMPG-CITY\tMPG-HWY\tPRICE',
 'subaru\tgas\tstd\ttwo\thatchback\tfwd\tfour\t69\t4900\t31\t36\t5118',
 'chevrolet\tgas\tstd\ttwo\thatchback\tfwd\tthree\t48\t5100\t47\t53\t5151',
 'mazda\tgas\tstd\ttwo\thatchback\tfwd\tfour\t68\t5000\t30\t31\t5195',
 'toyota\tgas\tstd\ttwo\thatchback\tfwd\tfour\t62\t4800\t35\t39\t5348']

In [27]:
toyota = carro.filter(lambda x:'toyota' in x)

In [28]:
toyota.take(5)

['toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348',
 'toyota,gas,std,two,hatchback,fwd,four,62,4800,31,38,6338',
 'toyota,gas,std,four,hatchback,fwd,four,62,4800,31,38,6488',
 'toyota,gas,std,four,wagon,fwd,four,62,4800,31,37,6918',
 'toyota,gas,std,four,sedan,fwd,four,70,4800,30,37,6938']

In [29]:
toyota.count()

32

In [30]:
with open('carros_v2.csv','w') as car:
    car.write('\n'.join(carro.collect()))

#### Set

In [31]:
rdd1 = sc.parallelize(['Big Data','Data Science','Analytics','Visualization'])
rdd2 = sc.parallelize(['Big Data','R','Python','Scala'])

In [32]:
for unions in rdd1.union(rdd2).distinct().collect():
    print(unions)

Analytics
Visualization
Big Data
Python
Data Science
R
Scala


In [33]:
for intersects in rdd1.intersection(rdd2).distinct().collect():
    print(intersects)

Big Data


In [34]:
names1 = sc.parallelize(('banana','uva','laranja')).map(lambda a: (a,1))
names2 = sc.parallelize(('laranja','abacaxi','manga')).map(lambda a: (a,1))

In [35]:
names1.join(names2).collect()

[('laranja', (1, 1))]

In [36]:
names1.leftOuterJoin(names2).collect()

[('banana', (1, None)), ('laranja', (1, 1)), ('uva', (1, None))]

In [37]:
numeros1 = [124,901,652,102,397,124,901,652]
rddnum = sc.parallelize(numeros1)
for num in rddnum.distinct().collect():
    print(num)

124
652
102
901
397


#### Transformações

In [38]:
carro.take(4)

['MAKE,FUELTYPE,ASPIRE,DOORS,BODY,DRIVE,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE',
 'subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118',
 'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151',
 'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195']

In [39]:
def limpardd(autostr):
    #Verifica a indexacao
    if isinstance(autostr,int):
        return autostr

    #Separa cada indice pela virgula
    attlist = autostr.split(',')

    #Converte o numero de portas para int
    if attlist[3] == 'two':
        attlist[3] = '2'
    elif attlist[3] == 'four':
        attlist[3] = '4'

    #Converte o modelo do carro para upper
    attlist[5] = attlist[4].upper()

    return ','.join(attlist)

In [40]:
carro_limpo = carro.map(limpardd)

In [41]:
carro_limpo.take(4)

['MAKE,FUELTYPE,ASPIRE,DOORS,BODY,BODY,CYLINDERS,HP,RPM,MPG-CITY,MPG-HWY,PRICE',
 'subaru,gas,std,2,hatchback,HATCHBACK,four,69,4900,31,36,5118',
 'chevrolet,gas,std,2,hatchback,HATCHBACK,three,48,5100,47,53,5151',
 'mazda,gas,std,2,hatchback,HATCHBACK,four,68,5000,30,31,5195']

#### Ações

In [42]:
#soma de valores
lista = [124,901,652,102,397,124,901,652]
lstRDD = sc.parallelize(lista)
lstRDD.collect()
lstRDD.reduce(lambda x,y: x + y)

3853

In [44]:
#linha com menor length
carro.reduce(lambda x,y: x if len(x) < len(y) else y)

'bmw,gas,std,two,sedan,rwd,six,182,5400,16,22,41315'

In [45]:
#criando uma função para redução
def getMPG(autoStr):
    if isinstance(autoStr,int):
        return autoStr
    attList = autoStr.split(',')
    if attList[9].isdigit():
        return int(attList[9])
    else:
        return 0

In [46]:
#Média de MPG
media_mpg = round(carro.reduce(lambda x,y:getMPG(x) + getMPG(y))/(carro.count()-1),2)
media_mpg

25.15

In [47]:
times = sc.parallelize(('Flamengo','Vasco','Botafogo','Fluminense','Palmeiras','Bahia'))
times.takeSample(True,3)

['Bahia', 'Vasco', 'Vasco']

In [48]:
times = sc.parallelize(('Flamengo','Vasco','Botafogo','Fluminense','Palmeiras','Bahia','Bahia','Flamengo'))
times.map(lambda k: (k,1)).countByKey().items()

dict_items([('Flamengo', 2), ('Vasco', 1), ('Botafogo', 1), ('Fluminense', 1), ('Palmeiras', 1), ('Bahia', 2)])

In [49]:
carro.saveAsTextFile('data/carro.txt')

#### Uber

Esse conjunto de dados contém dados de mais de 4,5 milhões de captações Uber na cidade de Nova york de abril a setembro de 2014 e 14,3 milhões de captações Uber de janeiro a junho de 2015. Dados em nível de viagem sobre 10 outras empresas  de veículos de aluguel (FHV) bem como dados agregados para 329 empresas de FHV, também estão incluídos. Todos os arquivos foram recebidos em 3 de agosto, 15 de setembro e 22 de setembro de 2015.

1. Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?
2. Qual o total de veiculos que passaram pela base B02617?
3. Qual o total de corridas por base? Apresente de forma descrescente

In [63]:
import wget
from pyspark.sql.functions import col, sum

In [51]:
wget.download('https://raw.githubusercontent.com/fivethirtyeight/uber-tlc-foil-response/master/Uber-Jan-Feb-FOIL.csv','./uber.csv')

'./uber.csv'

In [52]:
df = spark.read.csv('uber.csv',inferSchema=True,header=True)

In [53]:
df.show(10)

+-----------------------+--------+---------------+-----+
|dispatching_base_number|    date|active_vehicles|trips|
+-----------------------+--------+---------------+-----+
|                 B02512|1/1/2015|            190| 1132|
|                 B02765|1/1/2015|            225| 1765|
|                 B02764|1/1/2015|           3427|29421|
|                 B02682|1/1/2015|            945| 7679|
|                 B02617|1/1/2015|           1228| 9537|
|                 B02598|1/1/2015|            870| 6903|
|                 B02598|1/2/2015|            785| 4768|
|                 B02617|1/2/2015|           1137| 7065|
|                 B02512|1/2/2015|            175|  875|
|                 B02682|1/2/2015|            890| 5506|
+-----------------------+--------+---------------+-----+
only showing top 10 rows



In [56]:
df.select('dispatching_base_number').distinct().show()

+-----------------------+
|dispatching_base_number|
+-----------------------+
|                 B02512|
|                 B02598|
|                 B02682|
|                 B02765|
|                 B02617|
|                 B02764|
+-----------------------+



In [57]:
df.select('dispatching_base_number').distinct().count()

6

In [65]:
df.where(col('dispatching_base_number')=='B02617').select(sum(col('active_vehicles'))).show()

+--------------------+
|sum(active_vehicles)|
+--------------------+
|               79758|
+--------------------+



In [68]:
df.groupBy('dispatching_base_number').agg(sum('trips').alias('total')).orderBy(col('total').desc()).show()


+-----------------------+-------+
|dispatching_base_number|  total|
+-----------------------+-------+
|                 B02764|1914449|
|                 B02617| 725025|
|                 B02682| 662509|
|                 B02598| 540791|
|                 B02765| 193670|
|                 B02512|  93786|
+-----------------------+-------+



Uber RDD

In [1]:
from pandas import read_csv

In [2]:
uberfile = read_csv('uber.csv')

In [3]:
type(uberfile)

pandas.core.frame.DataFrame

In [4]:
uberfile.head(10)

,dispatching_base_number,date,active_vehicles,trips
0,B02512,1/1/2015,190,1132
1,B02765,1/1/2015,225,1765
2,B02764,1/1/2015,3427,29421
3,B02682,1/1/2015,945,7679
4,B02617,1/1/2015,1228,9537
5,B02598,1/1/2015,870,6903
6,B02598,1/2/2015,785,4768
7,B02617,1/2/2015,1137,7065
8,B02512,1/2/2015,175,875
9,B02682,1/2/2015,890,5506


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [15]:
spark = SparkSession.builder\
    .master('local[*]')\
    .getOrCreate()

sc = spark.sparkContext
sqlcontext = SQLContext(sc)

/home/vale/.local/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [17]:
uberdf = sqlcontext.createDataFrame(uberfile)

In [18]:
type(uberdf)

pyspark.sql.dataframe.DataFrame

In [19]:
uberdd = sc.textFile('uber.csv')

In [20]:
type(uberdd)

pyspark.rdd.RDD

In [21]:
uberdd.count()

355

In [22]:
uberdd.first()

'dispatching_base_number,date,active_vehicles,trips'

In [23]:
uberrow  = uberdd.map(lambda line: line.split(","))

In [24]:
type(uberrow)

pyspark.rdd.PipelinedRDD

In [25]:
uberrow.map(lambda linha:linha[0]).distinct().count() -1

6

In [26]:
uberrow.map(lambda linha:linha[0]).distinct().collect()

['dispatching_base_number',
 'B02765',
 'B02682',
 'B02598',
 'B02512',
 'B02764',
 'B02617']

In [27]:
uberrow.filter(lambda linha:  "B02617" in linha).count()

59

In [28]:
b02617rdd = uberrow.filter(lambda linha: "B02617" in linha)

In [29]:
#Total de dias em que o numero de corridas foi superior a 16000
b02617rdd.filter(lambda linha: int(linha[3]) > 16000).count()


4

In [30]:
#Dias em que o total de corridas foi superior a 16000
b02617rdd.filter(lambda linha: int(linha[3]) > 16000).collect()

[['B02617', '2/13/2015', '1590', '16996'],
 ['B02617', '2/14/2015', '1486', '16999'],
 ['B02617', '2/20/2015', '1574', '16856'],
 ['B02617', '2/21/2015', '1443', '16098']]

In [31]:
uberrdd = sc.textFile('uber.csv').filter(lambda line: "base" not in line).map(lambda line:line.split(','))

In [32]:
uberrdd.map(lambda kp: (kp[0], int(kp[3]))).reduceByKey(lambda k,v: k+v).collect()

[('B02765', 193670),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02512', 93786),
 ('B02764', 1914449),
 ('B02617', 725025)]

In [33]:
uberrdd.map(lambda kp: (kp[0], int(kp[3]))).reduceByKey(lambda k,v: k+v).takeOrdered(10, key= lambda x: -x[1])

[('B02764', 1914449),
 ('B02617', 725025),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02765', 193670),
 ('B02512', 93786)]